In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
import gc
import os
import matplotlib.pyplot as plt
import seaborn as sns 
import lightgbm as lgb
import itertools
import pickle, gzip
import glob
from sklearn.preprocessing import StandardScaler
from tsfresh.feature_extraction import extract_features
np.warnings.filterwarnings('ignore')
import dask.dataframe as dd
import missingno as msno
from pandasql import sqldf
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.model_selection import KFold
import matplotlib.gridspec as gridspec
from sklearn import preprocessing
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

In [2]:
#Always seed the randomness of this universe
np.random.seed(51)

In [3]:
%%time
pred1 = dd.read_csv('test_pred_40.csv')
pred2 = dd.read_csv('test_pred_46.csv')
pred1 = pred1.compute()
pred2 = pred2.compute()
print(pred1.shape,pred2.shape)

(3492890, 16) (3492890, 16)
CPU times: user 32.9 s, sys: 3.09 s, total: 36 s
Wall time: 7.5 s


In [4]:
%%time
pred1 = pred1.reset_index(drop=True)
pred2 = pred2.reset_index(drop=True)

CPU times: user 197 ms, sys: 272 ms, total: 470 ms
Wall time: 468 ms


In [5]:
pred1['object_id'].equals(pred2['object_id'])

True

In [6]:
%%time
test_ensemble = pd.DataFrame()

CPU times: user 284 µs, sys: 37 µs, total: 321 µs
Wall time: 325 µs


In [7]:
coef_lgb = 0.75
coef_mlp = 0.25

In [8]:
%%time
test_ensemble = pred1.iloc[:,1:]*coef_lgb + pred2.iloc[:,1:]*coef_mlp

CPU times: user 370 ms, sys: 585 ms, total: 955 ms
Wall time: 952 ms


In [9]:
print(test_ensemble.shape)

(3492890, 15)


In [10]:
test_ensemble['object_id'] = pred1['object_id']
print(test_ensemble.shape)

(3492890, 16)


In [11]:
test_ensemble = test_ensemble[list(pred1.columns)]

In [12]:
test_ensemble.tail()

,object_id,class_6,class_15,class_16,class_42,class_52,class_53,class_62,class_64,class_65,class_67,class_88,class_90,class_92,class_95,class_99
3492885,130787966,0.000162,0.344736,0.000323,0.189512,0.118729,0.000369,0.118128,0.000200,0.001354,0.014931,0.001784,0.209010,0.000236,0.000527,0.243964
3492886,130787971,0.000175,0.019250,0.000141,0.058373,0.168100,0.000214,0.020772,0.344966,0.000329,0.039863,0.000655,0.340776,0.000190,0.006197,0.232846
3492887,130787974,0.000600,0.021537,0.000119,0.833345,0.041256,0.000285,0.081527,0.000036,0.000130,0.001254,0.000336,0.017344,0.000179,0.002053,0.107819
3492888,130788053,0.000053,0.736005,0.000070,0.145347,0.003047,0.000097,0.009522,0.073570,0.000368,0.000282,0.000151,0.030912,0.000088,0.000489,0.156390
3492889,130788054,0.000184,0.099099,0.000269,0.426298,0.158324,0.000447,0.054113,0.000084,0.000596,0.019611,0.001077,0.238600,0.000596,0.000703,0.237256


In [13]:
%%time
test_ensemble.to_csv('test_pred_47.csv',index=False)

CPU times: user 1min 31s, sys: 1.24 s, total: 1min 32s
Wall time: 1min 32s


In [14]:
test_ensemble['object_id'].equals(pred1['object_id'])

True

In [15]:
!kaggle competitions submit -c PLAsTiCC-2018 -f test_pred_47.csv -m "Ensemble_LGB_MLP_075_025"

100%|██████████████████████████████████████| 1.06G/1.06G [05:02<00:00, 3.78MB/s]
Successfully submitted to PLAsTiCC Astronomical Classification